<h6>Project Title</h6>
"Predicting Heatwave Intensity and Duration Using Historical Climate Data"
<h6>Problem Statement</h6>
Heatwaves are becoming more frequent and severe due to climate change, causing health hazards, environmental stress, and socio-economic impacts. Currently, there is limited ability to accurately predict the intensity and duration of upcoming heatwaves in different cities, making it difficult for authorities and citizens to prepare effectively.
<h6>Project Description</h6>
This project uses historical climate data (temperature, apparent temperature, precipitation, wind speed) to predict heatwave intensity and duration using machine learning models. Classification models (Decision Tree, Random Forest) are used for intensity, and regression models predict duration.

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import joblib

In [7]:
# Importing Dataset
df = pd.read_excel(r'india_2004_2024_daily_weather.xlsx')
df.head()

,city,date,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,wind_speed_10m_max
0,Delhi,2004-01-01,17.6,5.0,16.9,3.5,0.0,0.0,8.8
1,Delhi,2004-01-02,17.8,5.3,16.0,3.4,0.0,0.0,10.8
2,Delhi,2004-01-03,16.7,6.3,14.6,4.1,0.0,0.0,12.2
3,Delhi,2004-01-04,15.8,4.2,13.3,1.5,0.0,0.0,12.3
4,Delhi,2004-01-05,15.4,5.2,14.3,3.1,0.0,0.0,9.6


In [8]:
# Basic Info
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89859 entries, 0 to 89858
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   city                      89859 non-null  object        
 1   date                      89859 non-null  datetime64[ns]
 2   temperature_2m_max        89859 non-null  float64       
 3   temperature_2m_min        89859 non-null  float64       
 4   apparent_temperature_max  89859 non-null  float64       
 5   apparent_temperature_min  89859 non-null  float64       
 6   precipitation_sum         89859 non-null  float64       
 7   rain_sum                  89859 non-null  float64       
 8   wind_speed_10m_max        89859 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 6.2+ MB
None
       temperature_2m_max  temperature_2m_min  apparent_temperature_max  \
count        89859.000000        89859.000000              

<h6>Observation</h6>
we have imported the dataset
<ul>
    <li>There are 89859 rows,9 columns</li>
    <li>There are No null values</li>
    <li>Identified the datatypes</li>
</ul>